<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/05_pytorch_going_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### O CONTEUDO EM INGLES EH COPIA,  em portugues o feito neste modulo
#### Copia ja q este modulo eh uma copia do passado, porem utilizando scripts python

```
pytorch_project/
├── pytorch_project/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```



* data_setup.py - um arquivo para preparar os dados (e baixar dados, se necessário).
* engine.py - um arquivo contendo várias funções de treinamento.
* model_builder.py ou model.py - um arquivo para criar um modelo PyTorch.
* train.py - um arquivo para utilizar todos os outros arquivos e treinar um modelo PyTorch alvo.
* utils.py - um arquivo dedicado a funções utilitárias úteis.

What we're going to cover
By the end of this notebook you should finish with a directory structure of:



```
going_modular/
├── going_modular/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model_builder.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
│   └── 05_going_modular_script_mode_tinyvgg_model.pth
└── data/
    └── pizza_steak_sushi/
        ├── train/
        │   ├── pizza/
        │   │   ├── image01.jpeg
        │   │   └── ...
        │   ├── steak/
        │   └── sushi/
        └── test/
            ├── pizza/
            ├── steak/
            └── sushi/
```


Using this directory structure, you should be able to train a model from within a notebook with the command:

```!python going_modular/train.py```

Or from the command line with:

```python going_modular/train.py```

In essence, we will have turned our helpful notebook code into reusable modular code.

# 0. Creating a folder for storing Python scripts
Since we're going to be creating Python scripts out of our most useful code cells, let's create a folder for storing those scripts.

We'll call the folder going_modular and create it using Python's os.makedirs() method.

In [1]:
import os

os.makedirs("going_modular", exist_ok=True)

# 1. Get data
We're going to start by downloading the same data we used in notebook 04, the pizza_steak_sushi dataset with images of pizza, steak and sushi.

In [2]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")


data/pizza_steak_sushi directory exists.
Unzipping pizza, steak, sushi data...


In [3]:

# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

# 2. Create Datasets and DataLoaders
Let's turn our data into PyTorch Dataset's and DataLoader's and find out a few useful attributes from them such as classes and their lengths.

In [4]:
from torchvision import datasets, transforms

# Create simple transform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [5]:
# Get class names as a list
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [6]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [7]:
# Check the lengths
len(train_data), len(test_data)

(225, 75)

In [8]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7d7642bba3e0>,
 <torch.utils.data.dataloader.DataLoader at 0x7d7642bbabc0>)

In [9]:
# Check out single image size/shape
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


## 2.1 Crianado o Dataset e Dataloaders com script

In [10]:
#Crinaod o diretorio going_modular
import os
os.mkdirs("going_modular")

AttributeError: module 'os' has no attribute 'mkdirs'

In [11]:
%%writefile going_modular/data_setup.py

"""
Contem as funcionalidades necessarias para criar Dataloaders para prblemas de classificacao de imagens
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
  train_dir: str,
  test_dir: str,
  transform: transforms.Compose,
  batch_size: int,
  num_workers=NUM_WORKERS,
):
  """Cria os DataLoaders de treino e teste

  Recebe um caminho para diretorio de treino e um para teste e transforma-os para Pytorch Datasets e depois em Pytorch DataLoaders

  Args:
    train_dir: caminho para o diretorio de treino.
    test_dir: caminho para o diretorio de teste.
    transform: torchvision transforms para transformar os dados de treino e teste.
    batch_size: numero de amostras por batch em cada dataloader
    num_workers: Numero inteiro de trabalhadores por DataLoader.

  Returns:
    Uma tupla de (tran_dataloader, test_dataloader, class_names)
    Aonde class_names eh uma lista das classes alvos
    Exemplo de uso:
      train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir,
        test_dir=pat/to/test_dir,
        transform=some_transform,
        batch_size=32,
        num_workers=NUM_WORKERS)
  """
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  #NOmes das classes
  class_names = train_data.classes

  #TRansformando em dataloaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True
  )

  return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


In [12]:
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=32)
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7d7706b39600>,
 ['pizza', 'steak', 'sushi'])

In [13]:
data_transform

Compose(
    Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)